In [73]:
import pandas as pd
from feature_calculation import build_biomarker_csv
import os

In [70]:
data = pd.read_csv("/Users/thomas.cong/Downloads/Voice_Data/W0000-NEU-CES-2157_Winterlight.csv")

subject_dict = data[["SUBJID", "X_DUID", "VISIT"]].drop_duplicates().set_index("X_DUID").to_dict()
for x in subject_dict["SUBJID"]:
    if subject_dict["VISIT"][x] == "V1":
        subject_dict["SUBJID"][x] += "OFF"
    else:
        subject_dict["SUBJID"][x] += "ON"
subject_dict = {x: subject_dict["SUBJID"][x] for x in subject_dict["SUBJID"]}
on_off_dict = {x: "OFF" if "OFF" in subject_dict[x] else "ON" for x in subject_dict}
subject_only = {x: subject_dict[x].split("O")[0] for x in subject_dict}

In [68]:
biomarkers_2157_with_clinical = pd.read_csv("./2157-Generated-Data/2157-Biomarkers.csv", index_col = "winterlight_id")
biomarkers_2157_with_clinical["subjid"] = biomarkers_2157_with_clinical.index.map(subject_dict)

In [91]:
UPDRS_Scores = pd.read_csv("/Users/thomas.cong/Downloads/Clinical_Data/2157 CES Clinical data from RUSH university/clinical-rush.csv")
UPDRS_Scores = pd.concat([UPDRS_Scores, pd.read_csv("/Users/thomas.cong/Downloads/Clinical_Data/2157 CES Clinical data from RUSH university/clinical-cumc.csv")])
UPDRS_Scores = UPDRS_Scores.pivot(index=["subjid", "state"], columns = ["mds_updrs_label"], values="mds_updrs_value")
UPDRS_Scores.reset_index(inplace=True)
UPDRS_Scores["subjid"] = UPDRS_Scores["subjid"] + UPDRS_Scores["state"]
UPDRS_Scores.drop(columns=["state"], inplace=True)
UPDRS_Scores.set_index("subjid", inplace=True)
UPDRSIII_Mapping = {x: sum(UPDRS_Scores.loc[x]) for x in UPDRS_Scores.index}
Speech_Mapping = {x: int(UPDRS_Scores.loc[x]["Speech"]) for x in UPDRS_Scores.index}
def add_clinical_info(biomarker_data_frame):
    biomarker_data_frame["subjid"] = biomarker_data_frame.index.map(subject_dict)
    biomarker_data_frame["UPDRSIII"] = biomarker_data_frame["subjid"].map(UPDRSIII_Mapping)
    biomarker_data_frame["Speech"] = biomarker_data_frame["subjid"].map(Speech_Mapping)
    if "task" in biomarker_data_frame.columns:
        biomarker_data_frame.drop(columns=["task"], inplace=True)
    try:
        biomarker_data_frame["Status"] = "O" + biomarker_data_frame["subjid"].str.split("O").str[1]
        biomarker_data_frame["Status"] = biomarker_data_frame["Status"].fillna("HC")
    except AttributeError:
        biomarker_data_frame["Status"] = "HC"
    return biomarker_data_frame
def normalize_biomarkers(biomarker_data_frame):
    for col in biomarker_data_frame.columns:
        if col:
            try:
                std = biomarkers_2157_with_clinical[col].std()
                mean = biomarkers_2157_with_clinical[col].mean()
                biomarkers_2157_with_clinical[col] = biomarkers_2157_with_clinical[col].apply(lambda x: max(min((x - mean) / std, 3), -3))
            except TypeError:
                pass
    return biomarker_data_frame
for file in os.listdir("2157-Generated-Data"):
    file_path = os.fsdecode(file)
    if not file_path.endswith(".csv"):
        continue
    info = "-".join(file_path.split("-")[1:])
    info = info.replace(".csv", "")
    print(info)
    biomarkers_2157_with_clinical = pd.read_csv("./2157-Generated-Data/" + file_path, index_col="winterlight_id")
    biomarkers_2157_with_clinical = add_clinical_info(biomarkers_2157_with_clinical)
    biomarkers_2157_with_clinical = normalize_biomarkers(biomarkers_2157_with_clinical)
    print(biomarkers_2157_with_clinical.shape)
    biomarkers_2157_with_clinical.to_csv(f"2157-Generated-Data/Clinical/2157-Clinical-{info}-normalized.csv")


Sustained-Vowel-Phonation
(63, 34)
Paragraph-Reading
(63, 34)
Paragraph-Recall
(21, 34)
Picture-Description
(126, 34)
Object-Naming
(62, 34)
Biomarkers
(524, 34)
DDK
(63, 34)
Phonemic-Fluency
(63, 34)
Semantic-Fleuncy
(63, 34)


In [ ]:
for col in biomarkers_2157_with_clinical.columns:
    if col != "UPDRSIII" and col != "Speech":
        try:
            std = biomarkers_2157_with_clinical[col].std()
            mean = biomarkers_2157_with_clinical[col].mean()
            biomarkers_2157_with_clinical[col] = biomarkers_2157_with_clinical[col].apply(lambda x: max(min((x - mean) / std, 3), -3))
        except TypeError:
            pass

In [66]:
biomarkers_2157_with_clinical.to_csv("./2157-Generated-Data/2157-Clinical-Biomarkers-Normalized.csv")